In [5]:
import requests
import pandas as pd

base_url = 'https://fantasy.premierleague.com/api/'

In [6]:
get_info = requests.get(f"{base_url}/bootstrap-static/").json()
element = pd.DataFrame(get_info['elements'])

In [7]:
pd.set_option('display.max_columns', None)
element.head()

,chance_of_playing_next_round,chance_of_playing_this_round,code,cost_change_event,cost_change_event_fall,cost_change_start,cost_change_start_fall,dreamteam_count,element_type,ep_next,ep_this,event_points,first_name,form,id,in_dreamteam,news,news_added,now_cost,photo,points_per_game,second_name,selected_by_percent,special,squad_number,status,team,team_code,total_points,transfers_in,transfers_in_event,transfers_out,transfers_out_event,value_form,value_season,web_name,minutes,goals_scored,assists,clean_sheets,goals_conceded,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,saves,bonus,bps,influence,creativity,threat,ict_index,starts,expected_goals,expected_assists,expected_goal_involvements,expected_goals_conceded,influence_rank,influence_rank_type,creativity_rank,creativity_rank_type,threat_rank,threat_rank_type,ict_index_rank,ict_index_rank_type,corners_and_indirect_freekicks_order,corners_and_indirect_freekicks_text,direct_freekicks_order,direct_freekicks_text,penalties_order,penalties_text,expected_goals_per_90,saves_per_90,expected_assists_per_90,expected_goal_involvements_per_90,expected_goals_conceded_per_90,goals_conceded_per_90,now_cost_rank,now_cost_rank_type,form_rank,form_rank_type,points_per_game_rank,points_per_game_rank_type,selected_rank,selected_rank_type,starts_per_90,clean_sheets_per_90
0,100.0,100.0,84450,0,0,-2,2,1,3,2.2,2.7,0,Granit,2.2,3,False,,2023-04-22T08:30:07.774962Z,48,84450.jpg,3.7,Xhaka,2.4,False,None,a,1,3,119,889085,341,810532,620,0.5,24.8,Xhaka,2617,5,5,11,32,0,0,0,4,0,0,11,524,571.6,608.4,435.0,161.6,31,2.94,3.53,6.47,32.91,62,24,28,21,65,41,38,26,NaN,,2.0,,NaN,,0.10,0.0,0.12,0.22,1.13,1.10,265,158,129,64,73,33,120,40,1.07,0.38
1,0.0,0.0,153256,0,0,-4,4,1,3,0.0,0.0,0,Mohamed,0.0,4,False,Knee injury - Unknown return date,2023-01-26T10:00:07.778526Z,41,153256.jpg,1.2,Elneny,0.6,False,None,i,1,3,6,200079,42,277417,99,0.0,1.5,Elneny,111,0,0,0,2,0,0,0,0,0,0,0,27,4.6,5.4,0.0,1.1,1,0.00,0.04,0.04,1.29,498,226,454,220,585,231,498,226,NaN,,NaN,,NaN,,0.00,0.0,0.03,0.03,1.05,1.62,625,325,519,189,435,191,246,79,0.81,0.00
2,NaN,NaN,156074,0,0,-3,3,0,2,1.8,2.3,0,Rob,1.8,5,False,,None,42,156074.jpg,1.5,Holding,0.2,False,None,a,1,3,20,20231,296,23202,87,0.4,4.8,Holding,557,1,0,0,13,0,0,0,0,0,0,0,117,149.6,10.3,54.0,21.4,6,0.32,0.15,0.47,10.85,319,127,440,156,327,111,383,137,NaN,,NaN,,NaN,,0.05,0.0,0.02,0.07,1.75,2.10,589,163,163,49,400,142,367,138,0.97,0.00
3,100.0,100.0,167199,0,0,-3,3,0,3,1.2,1.7,0,Thomas,1.2,6,False,,2023-03-31T15:30:08.770925Z,47,167199.jpg,2.8,Partey,0.4,False,None,a,1,3,79,108053,88,139491,121,0.3,16.8,Partey,2256,3,0,10,25,0,0,0,4,0,0,6,428,492.0,404.3,232.0,113.0,26,2.59,1.97,4.56,30.08,94,39,69,56,144,87,88,62,NaN,,NaN,,NaN,,0.10,0.0,0.08,0.18,1.20,1.00,310,186,236,111,183,74,278,91,1.04,0.40
4,100.0,100.0,184029,0,0,1,-1,4,3,4.5,5.0,0,Martin,4.5,7,True,,2023-03-09T22:30:08.763464Z,66,184029.jpg,5.6,Ødegaard,26.2,False,None,a,1,3,180,5988804,4157,4798218,11887,0.7,27.3,Ødegaard,2723,12,8,11,35,0,0,0,4,0,0,25,697,826.2,1001.5,806.0,263.6,32,8.73,7.11,15.84,34.04,13,5,8,6,20,10,8,6,3.0,,1.0,,NaN,,0.29,0.0,0.23,0.52,1.13,1.16,53,27,27,13,10,6,13,7,1.06,0.36


In [8]:
element['full_name'] = element['first_name'] + ' ' + element['web_name']
players = element[['id', 'full_name', 'web_name', 'team', 'element_type', 'total_points', 'selected_by_percent', 'now_cost', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 'own_goals', 'penalties_saved', 'penalties_missed', 'yellow_cards', 'red_cards', 'influence', 'creativity', 'threat', 'saves', 'form']]

In [12]:
teams = pd.DataFrame(get_info['teams'])
clubs = teams[['id','name', 'short_name']]
position = pd.DataFrame(get_info['element_types'])
position = position[['id', 'singular_name', 'singular_name_short']]
df1 = pd.merge(players, clubs, left_on='team', right_on='id')
df1 = pd.merge(df1, position, left_on = 'element_type', right_on = 'id')
df1.drop(["team", "id", "element_type"], axis=1, inplace=True)
df1 = df1.rename(columns = {'name': 'team', 'id_x': 'id' , 'web_name': 'shirt_name', 'singular_name' : 'position', 'singular_name_short': 'position_short', 'id_y': 'team_code'})
df1['now_cost'] = df1['now_cost'].apply(lambda x: float(x)/10)

In [13]:
df1['selected_by_percent'] = df1['selected_by_percent'].apply(lambda x: float(x))
df1['form'] = df1['form'].apply(lambda x: float(x))
df1['influence'] = df1['influence'].apply(lambda x: float(x))
df1['creativity'] = df1['creativity'].apply(lambda x: float(x))
df1['threat'] = df1['threat'].apply(lambda x: float(x))

In [14]:
sort_df = df1.sort_values(by = "total_points", ascending = False)
sort_fwd = sort_df[sort_df['position_short'] == 'FWD']
topfwd = sort_fwd.head(10)
topfwd

,id,full_name,shirt_name,total_points,selected_by_percent,now_cost,minutes,goals_scored,assists,clean_sheets,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,influence,creativity,threat,saves,form,team_code,team,short_name,position,position_short
642,318,Erling Haaland,Haaland,249,82.8,12.4,2404,34,8,11,0,0,0,5,0,1303.6,320.4,1650.0,0,7.7,13,Man City,MCI,Forward,FWD
664,427,Harry Kane,Kane,225,29.2,11.4,3046,25,8,10,0,0,1,5,0,1108.0,640.4,1591.0,0,7.2,18,Spurs,TOT,Forward,FWD
600,80,Ivan Toney,Toney,180,31.6,7.5,2863,20,4,10,0,0,1,9,0,920.8,410.4,1239.0,0,5.0,4,Brentford,BRE,Forward,FWD
589,40,Ollie Watkins,Watkins,161,20.4,7.6,2773,14,7,12,0,0,0,2,0,782.2,371.2,1213.0,0,6.6,2,Aston Villa,AVL,Forward,FWD
649,356,Callum Wilson,Wilson,128,4.7,7.0,1543,15,5,9,0,0,0,3,0,525.8,217.8,817.0,0,7.7,15,Newcastle,NEW,Forward,FWD
594,66,Dominic Solanke,Solanke,123,3.9,5.6,2506,6,10,8,0,0,0,1,0,480.0,239.9,930.0,0,6.1,3,Bournemouth,BOU,Forward,FWD
601,95,Bryan Mbeumo,Mbeumo,119,3.5,5.7,2547,6,7,8,0,0,0,4,0,515.6,715.3,708.0,0,4.1,4,Brentford,BRE,Forward,FWD
655,394,Brennan Johnson,Johnson,110,1.8,5.6,2740,8,3,7,0,0,2,6,0,456.8,362.2,713.0,0,2.1,16,Nott'm Forest,NFO,Forward,FWD
587,28,Gabriel Jesus,Jesus,101,27.9,8.1,1632,9,6,7,0,0,0,5,0,535.4,332.5,1090.0,0,5.0,1,Arsenal,ARS,Forward,FWD
609,145,Kai Havertz,Havertz,94,3.1,7.4,2215,7,1,9,0,0,0,4,0,455.0,488.1,902.0,0,1.0,6,Chelsea,CHE,Forward,FWD


In [79]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# sort dataframe by selected_by_percent column in ascending order
topfwd = topfwd.sort_values(by="total_points", ascending=True)

# create subplots
fig = make_subplots(rows=1, cols=4, shared_yaxes=True, horizontal_spacing=0.03)

# add traces to subplots
fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["total_points"], orientation="h", name="Total Points in FPL"),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["now_cost"], orientation="h", name="Cost in million"),
    row=1, col=2
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["selected_by_percent"], orientation="h", name="Selected by %"),
    row=1, col=3
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["form"], orientation="h", name="Form"),
    row=1, col=4
)

# update layout
fig.update_layout(
    title="Forwards and different attributes in selection",
    xaxis=dict(title="Attributes to look at"),
    yaxis=dict(title="Player Names"),
    height=800,
    width=1200,
    margin=dict(l=50, r=50, t=100, b=50),
    showlegend=False
)

fig.update_xaxes(title="Total FPL Points", row=1, col=1)
fig.update_xaxes(title="Cost in million", row=1, col=2)
fig.update_xaxes(title="Selected by %", row=1, col=3)
fig.update_xaxes(title="Form", row=1, col=4)

# show plot
fig.show()


In [16]:
sort_mid = sort_df[sort_df['position_short'] == 'MID']
topmid = sort_mid.head(10)
topmid

,id,full_name,shirt_name,total_points,selected_by_percent,now_cost,minutes,goals_scored,assists,clean_sheets,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,influence,creativity,threat,saves,form,team_code,team,short_name,position,position_short
190,283,Mohamed Salah,Salah,196,34.1,13.0,2850,17,8,10,0,0,2,2,0,874.4,724.7,1410.0,0,5.2,12,Liverpool,LIV,Midfielder,MID
216,335,Marcus Rashford,Rashford,192,44.1,7.2,2563,16,7,13,0,0,0,2,0,759.6,367.8,1196.0,0,5.1,14,Man Utd,MUN,Midfielder,MID
7,19,Gabriel Martinelli,Martinelli,190,40.4,6.8,2660,15,8,13,0,0,0,3,0,830.2,814.0,1255.0,0,5.5,1,Arsenal,ARS,Midfielder,MID
6,13,Bukayo Saka,Saka,187,31.2,8.2,2785,13,12,11,0,0,1,6,0,924.6,1011.9,1053.0,0,2.8,1,Arsenal,ARS,Midfielder,MID
3,7,Martin Ødegaard,Ødegaard,180,26.2,6.6,2723,12,8,11,0,0,0,4,0,826.2,1001.5,806.0,0,4.5,1,Arsenal,ARS,Midfielder,MID
202,301,Kevin De Bruyne,De Bruyne,178,28.6,12.2,2255,7,18,10,0,0,0,1,0,865.6,1380.6,588.0,0,8.2,13,Man City,MCI,Midfielder,MID
234,369,Miguel Almirón,Almirón,145,12.5,5.4,2123,11,3,14,0,0,0,2,0,551.4,315.7,754.0,0,1.3,15,Newcastle,NEW,Midfielder,MID
67,107,Solly March,March,145,14.5,5.3,2660,6,10,10,0,0,0,2,0,766.8,892.1,920.0,0,3.7,5,Brighton,BHA,Midfielder,MID
215,333,Bruno Fernandes,Fernandes,145,7.2,9.4,2789,6,8,15,0,0,0,6,0,800.2,1225.5,777.0,0,3.7,14,Man Utd,MUN,Midfielder,MID
281,428,Son Son,Son,140,6.0,11.5,2543,10,5,8,0,0,0,2,0,635.2,746.9,808.0,0,6.3,18,Spurs,TOT,Midfielder,MID


In [80]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# sort dataframe by selected_by_percent column in ascending order
topfwd = topmid.sort_values(by="total_points", ascending=True)

# create subplots
fig = make_subplots(rows=1, cols=4, shared_yaxes=True, horizontal_spacing=0.03)

# add traces to subplots
fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["total_points"], orientation="h", name="Total Points in FPL"),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["now_cost"], orientation="h", name="Cost in million"),
    row=1, col=2
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["selected_by_percent"], orientation="h", name="Selected by %"),
    row=1, col=3
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["form"], orientation="h", name="Form"),
    row=1, col=4
)

# update layout
fig.update_layout(
    title="Midfielders and different attributes in selection",
    xaxis=dict(title="Attributes to look at"),
    yaxis=dict(title="Player Names"),
    height=800,
    width=1200,
    margin=dict(l=50, r=50, t=100, b=50),
    showlegend=False
)

fig.update_xaxes(title="Total FPL Points", row=1, col=1)
fig.update_xaxes(title="Cost in million", row=1, col=2)
fig.update_xaxes(title="Selected by %", row=1, col=3)
fig.update_xaxes(title="Form", row=1, col=4)

# show plot
fig.show()


In [82]:
sort_def = sort_df[sort_df['position_short'] == 'DEF']
topdef = sort_def.head(10)
topdef

,id,full_name,shirt_name,total_points,selected_by_percent,now_cost,minutes,goals_scored,assists,clean_sheets,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,influence,creativity,threat,saves,form,team_code,team,short_name,position,position_short
513,357,Kieran Trippier,Trippier,177,66.3,6.1,2892,1,7,15,0,0,0,5,0,792.6,1601.1,107.0,0,3.3,15,Newcastle,NEW,Defender,DEF
329,10,Benjamin White,White,139,15.2,5.0,2604,2,5,13,0,0,0,5,0,591.0,435.2,153.0,0,3.0,1,Arsenal,ARS,Defender,DEF
519,366,Fabian Schär,Schär,131,14.4,5.1,2757,1,3,14,0,0,0,5,0,577.0,228.3,413.0,0,3.6,15,Newcastle,NEW,Defender,DEF
331,16,Gabriel Gabriel,Gabriel,129,14.5,5.2,2965,3,0,12,0,0,0,4,0,692.0,111.1,380.0,0,0.8,1,Arsenal,ARS,Defender,DEF
342,39,Tyrone Mings,Mings,125,13.0,4.7,2790,1,3,12,1,0,0,4,0,600.8,82.0,115.0,0,5.9,2,Aston Villa,AVL,Defender,DEF
475,285,Trent Alexander-Arnold,Alexander-Arnold,124,26.4,7.6,2472,1,10,7,0,0,0,4,0,665.6,1045.6,203.0,0,3.5,12,Liverpool,LIV,Defender,DEF
378,526,Ben Mee,Mee,122,16.8,5.0,2909,3,1,10,0,0,0,2,0,669.6,70.0,247.0,0,2.1,4,Brentford,BRE,Defender,DEF
334,26,William Saliba,Saliba,117,16.8,4.9,2415,2,1,12,1,0,0,4,0,529.0,100.8,114.0,0,0.0,1,Arsenal,ARS,Defender,DEF
522,377,Sven Botman,Botman,114,6.6,4.5,2677,0,2,12,0,0,0,2,0,476.4,100.4,272.0,0,2.9,15,Newcastle,NEW,Defender,DEF
514,358,Dan Burn,Burn,111,4.3,4.5,2743,0,0,13,0,0,0,4,0,464.8,197.4,208.0,0,2.3,15,Newcastle,NEW,Defender,DEF


In [83]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# sort dataframe by selected_by_percent column in ascending order
topfwd = topdef.sort_values(by="total_points", ascending=True)

# create subplots
fig = make_subplots(rows=1, cols=4, shared_yaxes=True, horizontal_spacing=0.03)

# add traces to subplots
fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["total_points"], orientation="h", name="Total Points in FPL"),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["now_cost"], orientation="h", name="Cost in million"),
    row=1, col=2
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["selected_by_percent"], orientation="h", name="Selected by %"),
    row=1, col=3
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["form"], orientation="h", name="Form"),
    row=1, col=4
)

# update layout
fig.update_layout(
    title="Defenders and different attributes in selection",
    xaxis=dict(title="Attributes to look at"),
    yaxis=dict(title="Player Names"),
    height=800,
    width=1200,
    margin=dict(l=50, r=50, t=100, b=50),
    showlegend=False
)

fig.update_xaxes(title="Total FPL Points", row=1, col=1)
fig.update_xaxes(title="Cost in million", row=1, col=2)
fig.update_xaxes(title="Selected by %", row=1, col=3)
fig.update_xaxes(title="Form", row=1, col=4)

# show plot
fig.show()


In [84]:
sort_gkp = sort_df[sort_df['position_short'] == 'GKP']
topgkp = sort_gkp.head(10)
topgkp

,id,full_name,shirt_name,total_points,selected_by_percent,now_cost,minutes,goals_scored,assists,clean_sheets,own_goals,penalties_saved,penalties_missed,yellow_cards,red_cards,influence,creativity,threat,saves,form,team_code,team,short_name,position,position_short
692,81,David Raya,Raya,146,16.0,4.9,3060,0,0,10,1,0,0,1,0,1015.0,20.1,4.0,138,3.7,4,Brentford,BRE,Goalkeeper,GKP
738,376,Nick Pope,Pope,139,20.7,5.4,2901,0,0,13,0,1,0,3,1,586.0,10.0,0.0,78,4.0,15,Newcastle,NEW,Goalkeeper,GKP
757,478,José Sá,Sá,138,5.5,5.0,3060,0,0,10,0,2,0,2,0,823.6,0.0,0.0,102,4.5,20,Wolves,WOL,Goalkeeper,GKP
723,281,Alisson Alisson,Alisson,136,15.3,5.4,2970,0,1,11,0,1,0,0,0,865.0,10.1,0.0,96,2.1,12,Liverpool,LIV,Goalkeeper,GKP
732,327,David De Gea,De Gea,132,12.6,5.0,2880,0,0,15,0,0,0,0,0,632.4,13.0,0.0,83,4.0,14,Man Utd,MUN,Goalkeeper,GKP
681,31,Emiliano Martínez,Martínez,126,6.6,5.0,2779,0,0,11,1,1,0,6,0,768.6,23.0,0.0,89,5.4,2,Aston Villa,AVL,Goalkeeper,GKP
678,15,Aaron Ramsdale,Ramsdale,120,10.8,4.8,2970,0,0,12,0,0,0,1,0,668.0,0.0,0.0,82,1.3,1,Arsenal,ARS,Goalkeeper,GKP
712,2,Bernd Leno,Leno,118,3.5,4.6,2790,0,0,7,0,0,0,3,0,1011.4,10.0,0.0,126,2.1,9,Fulham,FUL,Goalkeeper,GKP
753,455,Lukasz Fabianski,Fabianski,111,1.2,4.9,2661,0,0,8,0,1,0,1,0,630.4,0.0,0.0,90,3.1,19,West Ham,WHU,Goalkeeper,GKP
727,307,Ederson Ederson,Ederson,108,15.2,5.4,2880,0,1,10,0,0,0,3,0,443.6,13.1,0.0,40,1.7,13,Man City,MCI,Goalkeeper,GKP


In [86]:
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go

# sort dataframe by selected_by_percent column in ascending order
topfwd = topgkp.sort_values(by="total_points", ascending=True)

# create subplots
fig = make_subplots(rows=1, cols=4, shared_yaxes=True, horizontal_spacing=0.03)

# add traces to subplots
fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["total_points"], orientation="h", name="Total Points in FPL"),
    row=1, col=1
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["now_cost"], orientation="h", name="Cost in million"),
    row=1, col=2
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["selected_by_percent"], orientation="h", name="Selected by %"),
    row=1, col=3
)

fig.add_trace(
    go.Bar(y=topfwd["full_name"], x=topfwd["form"], orientation="h", name="Form"),
    row=1, col=4
)

# update layout
fig.update_layout(
    title="Goalkeepers and different attributes in selection",
    xaxis=dict(title="Attributes to look at"),
    yaxis=dict(title="Player Names"),
    height=800,
    width=1200,
    margin=dict(l=50, r=50, t=100, b=50),
    showlegend=False
)

fig.update_xaxes(title="Total FPL Points", row=1, col=1)
fig.update_xaxes(title="Cost in million", row=1, col=2)
fig.update_xaxes(title="Selected by %", row=1, col=3)
fig.update_xaxes(title="Form", row=1, col=4)

# show plot
fig.show()


In [22]:
|from pandas._libs.tslibs.offsets import Minute

def get_fwd_mid_stats(df):
  p_stats = df[['shirt_name', 'goals_scored', 'assists', 'influence', 'creativity', 'threat']]
  p_stats = p_stats.set_index('shirt_name')
  return p_stats

def get_def_stats(df):
  p_stats = df[['shirt_name', 'clean_sheets', 'assists', 'influence', 'creativity', 'threat', 'goals_scored']]
  p_stats = p_stats.set_index('shirt_name')
  return p_stats

def get_gkp_stats(df):
  p_stats = df[['shirt_name', 'clean_sheets', 'influence', 'saves']]
  p_stats = p_stats.set_index('shirt_name')
  return p_stats

In [23]:
comp_fwd = df1[df1['shirt_name'].isin(['Haaland', 'Darwin', 'Toney'])]
pl_stats = get_fwd_mid_stats(comp_fwd)
for col in pl_stats:
  pl_stats[col] = pl_stats[col].apply(lambda x :x / max(df1[col]))
angles = [i / float(len(pl_stats.columns)) * 360 for i in range(len(pl_stats.columns))]
values0 = pl_stats.iloc[0].tolist()
values0 += values0[:1]
values1 = pl_stats.iloc[1].tolist()
values1 += values1[:1]
values2 = pl_stats.iloc[2].tolist()
values2 += values1[:1]
trace0 = go.Scatterpolar(
    r=values0,
    theta=angles,
    fill='toself',
    name=pl_stats.index[0]
)
trace1 = go.Scatterpolar(
    r=values1,
    theta=angles,
    fill='toself',
    name=pl_stats.index[1]
)
trace2 = go.Scatterpolar(
    r=values2,
    theta=angles,
    fill='toself',
    name=pl_stats.index[2]
)
layout = go.Layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            showticklabels = False
        ),
    angularaxis=dict(
            tickvals=angles,
            ticktext=pl_stats.columns,
            direction='clockwise'
        )
    ),
    title=f'Toney performance compared to other top players'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    width=500,  # set width to 400 pixels
    height=500  # set height to 400 pixels
)
fig.show()

In [24]:
comp_mid = df1[df1['shirt_name'].isin(['Martinelli', 'Fernandes', 'March'])]
pl_stats = get_fwd_mid_stats(comp_mid)
for col in pl_stats:
  pl_stats[col] = pl_stats[col].apply(lambda x :x / max(df1[col]))
angles = [i / float(len(pl_stats.columns)) * 360 for i in range(len(pl_stats.columns))]
values0 = pl_stats.iloc[0].tolist()
values0 += values0[:1]
values1 = pl_stats.iloc[1].tolist()
values1 += values1[:1]
values2 = pl_stats.iloc[2].tolist()
values2 += values1[:1]
trace0 = go.Scatterpolar(
    r=values0,
    theta=angles,
    fill='toself',
    name=pl_stats.index[0]
)
trace1 = go.Scatterpolar(
    r=values1,
    theta=angles,
    fill='toself',
    name=pl_stats.index[1]
)
trace2 = go.Scatterpolar(
    r=values2,
    theta=angles,
    fill='toself',
    name=pl_stats.index[2]
)
layout = go.Layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            showticklabels = False
        ),
    angularaxis=dict(
            tickvals=angles,
            ticktext=pl_stats.columns,
            direction='clockwise'
        )
    ),
    title=f'March performance compared to other top players'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    width=500,  # set width to 400 pixels
    height=500  # set height to 400 pixels
)
fig.show()

In [25]:
comp_def = df1[df1['shirt_name'].isin(['Trippier', 'Van Dijk', 'Mings'])]
pl_stats = get_def_stats(comp_def)
for col in pl_stats:
  pl_stats[col] = pl_stats[col].apply(lambda x :x / max(df1[col]))
angles = [i / float(len(pl_stats.columns)) * 360 for i in range(len(pl_stats.columns))]
values0 = pl_stats.iloc[0].tolist()
values0 += values0[:1]
values1 = pl_stats.iloc[1].tolist()
values1 += values1[:1]
values2 = pl_stats.iloc[2].tolist()
values2 += values1[:1]
trace0 = go.Scatterpolar(
    r=values0,
    theta=angles,
    fill='toself',
    name=pl_stats.index[0]
)
trace1 = go.Scatterpolar(
    r=values1,
    theta=angles,
    fill='toself',
    name=pl_stats.index[1]
)
trace2 = go.Scatterpolar(
    r=values2,
    theta=angles,
    fill='toself',
    name=pl_stats.index[2]
)
layout = go.Layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            showticklabels = False
        ),
    angularaxis=dict(
            tickvals=angles,
            ticktext=pl_stats.columns,
            direction='clockwise'
        )
    ),
    title=f'Mings performance compared to other top players'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    width=500,  # set width to 400 pixels
    height=500  # set height to 400 pixels
)
fig.show()

In [26]:
comp_gkp = df1[df1['shirt_name'].isin(['Raya', 'Leno', 'Ederson'])]
pl_stats = get_gkp_stats(comp_gkp)
for col in pl_stats:
  pl_stats[col] = pl_stats[col].apply(lambda x :x / max(df1[col]))
angles = [i / float(len(pl_stats.columns)) * 360 for i in range(len(pl_stats.columns))]
values0 = pl_stats.iloc[0].tolist()
values0 += values0[:1]
values1 = pl_stats.iloc[1].tolist()
values1 += values1[:1]
values2 = pl_stats.iloc[2].tolist()
values2 += values1[:1]
trace0 = go.Scatterpolar(
    r=values0,
    theta=angles,
    fill='toself',
    name=pl_stats.index[0]
)
trace1 = go.Scatterpolar(
    r=values1,
    theta=angles,
    fill='toself',
    name=pl_stats.index[1]
)
trace2 = go.Scatterpolar(
    r=values2,
    theta=angles,
    fill='toself',
    name=pl_stats.index[2]
)
layout = go.Layout(
    polar=dict(
        radialaxis=dict(
            visible=True,
            range=[0, 1],
            showticklabels = False
        ),
    angularaxis=dict(
            tickvals=angles,
            ticktext=pl_stats.columns,
            direction='clockwise'
        )
    ),
    title=f'Leno performance compared to other top players'
)

fig = go.Figure(data=[trace0, trace1, trace2], layout=layout)
fig.update_layout(
    polar=dict(
        radialaxis=dict(visible=True)
    ),
    width=500,  # set width to 400 pixels
    height=500  # set height to 400 pixels
)
fig.show()

In [70]:
from datetime import datetime
import pytz

players = ['Toney', 'March', 'Mings']
club_code = {}
fig = go.Figure()
for player in players:
  player_id = df1.loc[df1['shirt_name'] == player, 'id'].values[0]
  player_club_code = df1.loc[df1['shirt_name'] == player, 'team_code'].values[0]
  player_club = df1.loc[df1['shirt_name'] == player, 'team'].values[0]
  club_code[player_club] = player_club_code
  url_history = f'https://fantasy.premierleague.com/api/element-summary/{player_id}/'
  history = requests.get(url_history).json()
  history = history['history']
  history = pd.DataFrame(history)
  history['kickoff_time'] = pd.to_datetime(history['kickoff_time'], utc=True)
  history_ = history[['total_points', 'kickoff_time']]
  date_limit = datetime(2023, 4, 30, tzinfo=pytz.UTC)
  history_ = history_[history_['kickoff_time'] <= date_limit]
  history_ = history_.groupby([pd.Grouper(key='kickoff_time', freq='M')]).sum()
  fig.add_trace(go.Scatter(x=history_.index, y=history_['total_points'], mode='lines+markers', name = player))

# Update layout
fig.update_layout( 
    title = 'Total Points Throughout the Season',
    xaxis_title='Months',
    yaxis_title='Total Points',
    width=800,
    height=600
)

# Show plot
fig.show()

In [78]:
import requests
import pandas as pd
import plotly.graph_objects as go

# retrieve data for Arsenal's upcoming fixtures from the FPL API
url = 'https://fantasy.premierleague.com/api/fixtures/'
response = requests.get(url)
fixtures = pd.DataFrame(response.json())
for team, team_code in club_code.items():
  arsenal_fixtures = fixtures[(fixtures['team_a'] == team_code) | (fixtures['team_h'] == team_code)]
  arsenal_fixtures['difficulties'] = arsenal_fixtures.apply(lambda x: x['team_a_difficulty'] if x['team_a'] == 1 else x['team_h_difficulty'], axis=1)
  arsenal_fixtures['opponent'] = arsenal_fixtures.apply(lambda x: x['team_h'] if x['team_a'] == 1 else x['team_a'], axis=1)
  arsenal_fixtures = arsenal_fixtures[arsenal_fixtures['finished'] == False]
  arsenal_fixtures['constant'] = 1
  
  fig = px.bar(arsenal_fixtures, 
              x='event', 
              y='constant', 
              color='difficulties', 
              color_continuous_scale='RdYlGn_r',
              title=f'{team} Fixtures Difficulty by Gameweek')
  fig.update_layout(xaxis={'title': 'Gameweek'},
                    yaxis=dict(tickvals=[]), 
                    coloraxis_colorbar=dict(title='Difficulty', 
                                            tickvals=[1, 2, 3, 4, 5], 
                                            ticktext=['Very Easy', 'Easy', 'Moderate', 'Difficult', 'Very Difficult']))
  fig.show()


<ipython-input-78-0cd24c653778>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-78-0cd24c653778>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-78-0cd24c653778>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-78-0cd24c653778>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



<ipython-input-78-0cd24c653778>:11: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-78-0cd24c653778>:12: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

